In [1]:
# Source:  https://www.youtube.com/watch?v=ApA7EVwSzg0
# Splinter doc: https://splinter.readthedocs.io/en/latest/finding.html
# Selenium list of options: 
#   https://peter.sh/experiments/chromium-command-line-switches/
#   https://chromium.googlesource.com/chromium/src/+/master/chrome/common/chrome_switches.cc


import time
import urllib
import requests
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.by import By
from config import tda_real_username, tda_real_password , tda_real_url, tda_Consumer_Key, tda_security_question_oldestnephew , tda_security_question_father_born , tda_security_question_first_pet, tda_security_question_meet_spouse, tda_access_token_file, tda_refresh_token_file 
import re


# define path to chrome driver
executable_path = {'executable_path':r'C:\Users\Owner\python_projects\chromedriver.exe'}

# Set some default behaviors for our browser
options = webdriver.ChromeOptions()

# Make sure the window is maximized
options.add_argument("--start-maximized")

# Make sure that notification are off
options.add_argument("--disable-notifications")

# TDA authorization section
# create a new browser object, by default it is firefox
tda_browser = Browser("chrome", **executable_path, headless = False, options = options)


# define the components of the url
method = 'GET'
url = tda_real_url
tda_account_id_code = tda_Consumer_Key + '@AMER.OAUTHAP'
payload = {'response_type':'code', 'redirect_uri':'http://localhost/test','client_id':tda_account_id_code}

# build the url
tda_built_url = requests.Request(method, url, params = payload).prepare()
tda_built_url = tda_built_url.url

# go to our url
tda_browser.visit(tda_built_url)

# Fill id element for username
tda_browser.find_by_id("username0").fill(tda_real_username)

# Fill id element for password
tda_browser.find_by_id("password").fill(tda_real_password)

# Click login button
tda_browser.find_by_id("accept").first.click()

# Click on "Can't get the text message?"" to make "Answer a Security Question" visible
tda_browser.find_by_text("Can't get the text message?").first.click()

# Click on "Answer a security question" to open security question page
tda_browser.find_by_name("init_secretquestion").first.click()

# Find the Question box to parse the security question
# Get all the elements available with tag name 'p'
elements = tda_browser.find_by_tag('p')
#join_parameter1=','
for e in elements:
    # The below block is also good to spot matched keywords in a string
    #keywords = ['oldest', 'nephew']
    #matched_keywords = set(e.text.split()) & set(keywords)
    #if matched_keywords:
    #    print(e.text)
    
    # If the keyword found, return as a non empty list, then anser the box
    if re.findall(r'oldest nephew', e.text):
        tda_browser.find_by_id("secretquestion0").fill(tda_security_question_oldestnephew)
    elif re.findall(r'father born', e.text):
        tda_browser.find_by_id("secretquestion0").fill(tda_security_question_father_born)
    elif re.findall(r'first pet', e.text):
        tda_browser.find_by_id("secretquestion0").fill(tda_security_question_first_pet)
    elif re.findall(r'spouse', e.text):
        tda_browser.find_by_id("secretquestion0").fill(tda_security_question_meet_spouse)
del e, elements    
time.sleep(1)
# click the continue button
tda_browser.find_by_id("accept").first.click()

# Trust This device screen will show
# Click on "Yes, trust this device" radio button
elements = tda_browser.find_by_tag('label')
for e in elements:
    if re.findall(r'Yes,', e.text):
        e.click()
        break
del e, elements

# Click "Save"
tda_browser.find_by_id("accept").first.click()

time.sleep(1)
# Screen asking to for this app to access real TD Ameritrade account will show
# Click Allow
tda_browser.find_by_id("accept").first.click()

# give it a sec to load
time.sleep(1)
new_tda_url = tda_browser.url

# grab the url and parse it
new_tda_authorization_code = urllib.parse.unquote(new_tda_url.split("code=")[1])

# Close the browser
tda_browser.quit()

new_tda_authorization_code

# define authorization URL with acccess token
tda_authorization_token_url = r'https://api.tdameritrade.com/v1/oauth2/token'

# define the headers
tda_headers = {'Content-Type':'application/x-www-form-urlencoded'}

# Here, the grant_type is the type of tokens with values: authorization_code or refresh_token

tda_payload = {'grant_type':'authorization_code',
               'access_type':'offline',
               'code':new_tda_authorization_code,
               'redirect_uri':'http://localhost/test',
               'client_id':tda_Consumer_Key               
               # Once refresh_token granted, comment out access_type, code, redirect_uri
               # Then fill out refresh_token below
               #'refresh_token':''
               
              }


# post the data to get token
tda_authReply = requests.post(tda_authorization_token_url, headers = tda_headers, data = tda_payload)

# convert json string to dictionary
tda_decoded_content = tda_authReply.json()
#tda_decoded_content


tda_access_token = tda_decoded_content['access_token']
tda_refresh_token = tda_decoded_content['refresh_token']
#print(tda_access_token)

# store tda_access_token and refresh_token in trading_db
with open(tda_access_token_file, 'w') as writeobj1, open(tda_refresh_token_file , 'w') as writeobj2:
    writeobj1.write(tda_access_token)
    writeobj2.write(tda_refresh_token)
    writeobj1.close()
    writeobj2.close()
    
    
    
    
#del tda_decoded_content, tda_access_token, tda_authReply, tda_payload, tda_headers, tda_browser, new_tda_authorization_code, new_tda_url
#del tda_built_url, tda_account_id_code, method, url, payload, executable_path
with open(tda_access_token_file, 'r') as readobj:
    tda_access_token = readobj.read()
    readobj.close()

    
    
    





In [2]:
tda_trading_headers = {'Authorization': "Bearer {}".format(tda_access_token)}
#tda_trading_headers

# ACCOUNT ENDPOINT - Query for a single account

# define an endpoint with a stock of your choice, MUST BE UPPER
tda_account_endpoint = r"https://api.tdameritrade.com/v1/accounts"

# make a request to retrieve account info
tda_account_info = requests.get(url = tda_account_endpoint, headers = tda_trading_headers)

# convert it to dictionary object
tda_account_info = tda_account_info.json()
display(tda_account_info)

tda_account_id = tda_account_info[0]['securitiesAccount']['accountId']
#tda_account_id

[{'securitiesAccount': {'type': 'CASH',
   'accountId': '270470060',
   'roundTrips': 0,
   'isDayTrader': False,
   'isClosingOnlyRestricted': False,
   'initialBalances': {'accruedInterest': 0.0,
    'cashAvailableForTrading': 0.0,
    'cashAvailableForWithdrawal': 0.0,
    'cashBalance': 0.0,
    'bondValue': 0.0,
    'cashReceipts': 0.0,
    'liquidationValue': 0.0,
    'longOptionMarketValue': 0.0,
    'longStockValue': 0.0,
    'moneyMarketFund': 0.0,
    'mutualFundValue': 0.0,
    'shortOptionMarketValue': 0.0,
    'shortStockValue': 0.0,
    'isInCall': False,
    'unsettledCash': 0.0,
    'cashDebitCallValue': 0.0,
    'pendingDeposits': 0.0,
    'accountValue': 0.0},
   'currentBalances': {'accruedInterest': 0.0,
    'cashBalance': 0.0,
    'cashReceipts': 0.0,
    'longOptionMarketValue': 0.0,
    'liquidationValue': 0.0,
    'longMarketValue': 0.0,
    'moneyMarketFund': 0.0,
    'savings': 0.0,
    'shortMarketValue': 0.0,
    'pendingDeposits': 0.0,
    'cashAvailableFor

In [3]:
## Create Saved Order
tda_order_headers = {'Authorization': "Bearer {}".format(tda_access_token),
                     "Content-Type":"application/json"
                     }
# define the endpiont for saved orders, including your account ID
tda_savedorders_endpoint = r"https://api.tdameritrade.com/v1/accounts/{}/savedorders".format(tda_account_id)

# define payload in JSON format
tda_savedorders_payload = {"session":"NORMAL",
                           "duration":'DAY',
                           "orderType":'LIMIT',
                           "price": "0.0000001",
                           "orderLegCollection": [{
                              "instruction": "BUY",
                              "quantity": 1,
                              "instrument": {
                              "symbol": "BYOC",
                              "assetType": "EQUITY"
                                            } 
                                                }],
                          }

# make a post , NOTE WE've CHANGED DATA TO JSON
tda_savedorders_content = requests.post(url = tda_savedorders_endpoint, json = tda_savedorders_payload, headers = tda_order_headers)

# show the status code, we want 200
tda_savedorders_content.status_code

200

In [4]:
#del tda_decoded_content, tda_access_token, tda_authReply, tda_payload, tda_headers, tda_browser, new_tda_authorization_code, new_tda_url
#del tda_built_url, tda_account_id_code, method, url, payload, executable_path
with open(tda_access_token_file, 'r') as readobj:
    tda_access_token = readobj.read()
    readobj.close()

    
    
# query list of saved orders
tda_query_savedorders_headers = {'Authorization': "Bearer {}".format(tda_access_token)}
tda_savedorders_endpoint = r"https://api.tdameritrade.com/v1/accounts/{}/savedorders".format(tda_account_id)

# Retrieve list of order via a GET
tda_savedorders_list = requests.get(url = tda_savedorders_endpoint, headers = tda_query_savedorders_headers)

# Show status code, we want 200
display(tda_savedorders_list.status_code)

# Show content of the saved orders
tda_savedorders_list = tda_savedorders_list.json()
display(tda_savedorders_list)

200

[{'session': 'NORMAL',
  'duration': 'DAY',
  'orderType': 'LIMIT',
  'complexOrderStrategyType': 'NONE',
  'price': 0.0,
  'orderLegCollection': [{'orderLegType': 'EQUITY',
    'legId': 1,
    'instrument': {'assetType': 'EQUITY', 'symbol': 'BYOC'},
    'instruction': 'BUY',
    'quantity': 1.0}],
  'orderStrategyType': 'SINGLE',
  'cancelable': True,
  'editable': True,
  'savedOrderId': 15070082,
  'savedTime': '2020-08-08T22:59:05+0000'}]

In [5]:
## NEED TO WRITE ORDERS IN A FILE THAT ALL PROCESS CAN ACCESS
# REMOVE SAVED ORDERS

#del tda_decoded_content, tda_access_token, tda_authReply, tda_payload, tda_headers, tda_browser, new_tda_authorization_code, new_tda_url
#del tda_built_url, tda_account_id_code, method, url, payload, executable_path
with open(tda_access_token_file, 'r') as readobj:
    tda_access_token = readobj.read()
    readobj.close()

# define a single order savedOrderId from list of orders
savedOrderId = "15022494"

tda_order_headers = {'Authorization': "Bearer {}".format(tda_access_token),
                     "Content-Type":"application/json"
                     }
# define the endpiont for saved orders, including your account ID
tda_savedorders_ID_endpoint = r"https://api.tdameritrade.com/v1/accounts/{}/savedorders/{}".format(tda_account_id, savedOrderId)

# delete / cancel saved order with delete method
tda_savedorders_content = requests.delete(url = tda_savedorders_ID_endpoint, headers = tda_order_headers)

# query for status of cancelation of saved orders
tda_savedorders_content.status_code

# delete all saved orders
for savedOrder in tda_savedorders_list:
    savedOrderId = savedOrder['savedOrderId']
    tda_savedorders_ID_endpoint = r"https://api.tdameritrade.com/v1/accounts/{}/savedorders/{}".format(tda_account_id, savedOrderId)
    tda_savedorders_content = requests.delete(url = tda_savedorders_ID_endpoint, headers = tda_order_headers)
    tda_savedorders_content.status_code
    # check status to make sure saved orders are deleted 1 by 1
    tda_savedorders_list = requests.get(url = tda_savedorders_endpoint, headers = tda_order_headers)
    display(tda_savedorders_list.status_code)
    tda_savedorders_list = tda_savedorders_list.json()
    display(tda_savedorders_list)

200

[]